# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [101]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [102]:
#發現有些fare是免錢的
df[df['Fare']==0].head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
179,3,"Leonard, Mr. Lionel",male,36.0,0,0,LINE,0.0,NaN,S
263,1,"Harrison, Mr. William",male,40.0,0,0,112059,0.0,B94,S
271,3,"Tornquist, Mr. William Henry",male,25.0,0,0,LINE,0.0,NaN,S
277,2,"Parkes, Mr. Francis ""Frank""",male,NaN,0,0,239853,0.0,NaN,S
302,3,"Johnson, Mr. William Cahoone Jr",male,19.0,0,0,LINE,0.0,NaN,S


In [103]:
#利用'Pclass','Sex','Age'去補fare=0時的值
cal_fare = df[df['Fare']>0]
cal_fare = cal_fare.groupby(['Pclass','Sex','Age'])['Fare'].mean().reset_index()
cal_fare.head()

,Pclass,Sex,Age,Fare
0,1,female,2.0,151.550000
1,1,female,14.0,120.000000
2,1,female,15.0,211.337500
3,1,female,16.0,61.293067
4,1,female,17.0,82.950000


In [104]:
#但發現仍有FARE是NA
pd.merge(df[df['Fare']==0], cal_fare, how='left', on=['Pclass','Sex','Age']).head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare_x,Cabin,Embarked,Fare_y
0,3,"Leonard, Mr. Lionel",male,36.0,0,0,LINE,0.0,NaN,S,13.772900
1,1,"Harrison, Mr. William",male,40.0,0,0,112059,0.0,B94,S,29.360400
2,3,"Tornquist, Mr. William Henry",male,25.0,0,0,LINE,0.0,NaN,S,8.521250
3,2,"Parkes, Mr. Francis ""Frank""",male,NaN,0,0,239853,0.0,NaN,S,NaN
4,3,"Johnson, Mr. William Cahoone Jr",male,19.0,0,0,LINE,0.0,NaN,S,8.617409


In [105]:
#目前處理的方式先移除0
train_Y = train_Y[df['Fare']>0]
df = df[df['Fare']>0]

In [106]:
# 票價(Fare)對性別(Sex)做群聚編碼
mean_df = df.groupby(['Sex'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Sex'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Sex'])['Fare'].median().reset_index()
max_df = df.groupby(['Sex'])['Fare'].max().reset_index()
min_df = df.groupby(['Sex'])['Fare'].min().reset_index()
count_df = df.groupby(['Sex'])['Fare'].count().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Sex'])
temp = pd.merge(temp, median_df, how='left', on=['Sex'])
temp = pd.merge(temp, max_df, how='left', on=['Sex'])
temp = pd.merge(temp, min_df, how='left', on=['Sex'])
temp = pd.merge(temp, count_df, how='left', on=['Sex'])
temp.columns = ['Sex', 'Fare_Sex_Mean', 'Fare_Sex_Mode', 'Fare_Sex_Median', 'Fare_Sex_Max', 'Fare_Sex_Min', 'Fare_Sex_Count']
temp

,Sex,Fare_Sex_Mean,Fare_Sex_Mode,Fare_Sex_Median,Fare_Sex_Max,Fare_Sex_Min,Fare_Sex_Count
0,female,44.479818,7.75,23.0000,512.3292,6.7500,314
1,male,26.205136,8.05,12.3125,512.3292,4.0125,562


In [107]:
df = pd.merge(df, temp, how='left', on=['Sex'])
df = df.drop(['Sex'] , axis=1)
df.head()

,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Sex_Mean,Fare_Sex_Mode,Fare_Sex_Median,Fare_Sex_Max,Fare_Sex_Min,Fare_Sex_Count
0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,26.205136,8.05,12.3125,512.3292,4.0125,562
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,44.479818,7.75,23.0000,512.3292,6.7500,314
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,44.479818,7.75,23.0000,512.3292,6.7500,314
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,44.479818,7.75,23.0000,512.3292,6.7500,314
4,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,26.205136,8.05,12.3125,512.3292,4.0125,562


In [108]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

11 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_Sex_Mean', 'Fare_Sex_Mode', 'Fare_Sex_Median', 'Fare_Sex_Max', 'Fare_Sex_Min', 'Fare_Sex_Count']



,Pclass,Age,SibSp,Parch,Fare,Fare_Sex_Mean,Fare_Sex_Mode,Fare_Sex_Median,Fare_Sex_Max,Fare_Sex_Min,Fare_Sex_Count
0,3,22.0,1,0,7.2500,26.205136,8.05,12.3125,512.3292,4.0125,562
1,1,38.0,1,0,71.2833,44.479818,7.75,23.0000,512.3292,6.7500,314
2,3,26.0,0,0,7.9250,44.479818,7.75,23.0000,512.3292,6.7500,314
3,1,35.0,1,0,53.1000,44.479818,7.75,23.0000,512.3292,6.7500,314
4,3,35.0,0,0,8.0500,26.205136,8.05,12.3125,512.3292,4.0125,562


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [109]:
# 沒有這四個新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['Fare_Sex_Mean', 'Fare_Sex_Mode', 'Fare_Sex_Median', 'Fare_Sex_Max', 'Fare_Sex_Min', 'Fare_Sex_Count'] , axis=1)

In [110]:
# 原始特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=10).mean()

0.7114976283564041

In [111]:
# 新特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=10).mean()

0.7830406936470478

In [112]:
print("有改善")

有改善
